In [ ]:
! pip install llama-index-tools-mcp llama-index python-dotenv llama-index-tools-wikipedia fastmcp -q


In [ ]:
from fastmcp import Client
from fastmcp.client import SSETransport

In [14]:
async with Client(SSETransport("http://localhost:8000/sse")) as client:
    tools = await client.list_tools()
tools

[Tool(name='create_hero_heroes__post', description='Create Hero\n\n### Responses:\n\n**200**: Successful Response (Success Response)\nContent-Type: application/json\n\n**Example Response:**\n```json\n{\n  "name": "Name"\n}\n```', inputSchema={'type': 'object', 'properties': {'id': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'title': 'id', 'type': 'integer'}, 'name': {'type': 'string', 'title': 'name', 'description': 'Name of the hero'}, 'real_name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'real_name', 'description': 'Real name of the hero', 'type': 'string'}, 'origin': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'origin', 'description': 'Place of origin', 'type': 'string'}, 'superpowers': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'superpowers', 'description': 'Comma-separated list of superpowers', 'type': 'string'}, 'team': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'team', 'description': 'Team or organization af

# Create agents

In [16]:
from llama_index.llms.openai import OpenAI
from dotenv import load_dotenv
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec
from llama_index.tools.wikipedia import WikipediaToolSpec
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.workflow import Context


In [17]:
load_dotenv()

llm = OpenAI(model="gpt-4o-mini")

## Search information from Wikipedia and save it to API Server

In [18]:
hero_mcp_client = BasicMCPClient("http://localhost:8000/sse")

hero_mcp_tool_spec = McpToolSpec(
    client=hero_mcp_client,
)

hero_tools = await hero_mcp_tool_spec.to_tool_list_async()
hero_tools

In [19]:
helper_mcp_client = BasicMCPClient("http://localhost:8001/sse")

helper_mcp_tool_spec = McpToolSpec(
    client=helper_mcp_client,
)

helper_tools = await helper_mcp_tool_spec.to_tool_list_async()
helper_tools

In [20]:
wiki_tool_spec = WikipediaToolSpec()
wiki_tools = wiki_tool_spec.to_tool_list()

In [21]:
collector_agent = FunctionAgent(
    tools=[
        *hero_tools,
        *wiki_tools,
        *helper_tools
    ],
    llm=llm,
    system_prompt="You are a helpful collector agent. You can use the tools to collect information from the Wikipedia and save it to the MCP server. You can also use the MCP server to get information from the tools. You can use the tools to collect information from the Wikipedia and save it to the MCP server. You can also use the MCP server to get information and use it to answer the user's question. If you use add for date comparison, you should use defined tools. If you don't know the answer and can not find it, say 'I don't know'.",
)

ctx = Context(collector_agent)  # type: ignore

In [27]:
response = await collector_agent.run("Find all information about the hero 'Superman' from Wikipedia and save it to the MCP server. Then, find all information about the hero 'Batman' from the MCP server.", ctx=ctx)
print("Response:")

print(str(response))

Response:
I have successfully saved all information about the hero **Superman** to the MCP server. Here are the details:

- **Name:** Superman
- **Real Name:** Kal-El
- **Origin:** Krypton
- **Team:** Justice League
- **First Appearance:** Action Comics #1, 1938
- **Superpowers:** Superhuman strength, invulnerability, flight, heat vision, x-ray vision, super speed, super breath
- **Biography:** Superman is a superhero created by writer Jerry Siegel and artist Joe Shuster, first appearing in the comic book Action Comics #1, published in America on April 18, 1938. Superman has been regularly published in American comic books since 1938, and has been adapted to other media including radio serials, novels, films, television shows, theater, and video games. Superman was born Kal-El, on the fictional planet Krypton. As a baby, his parents Jor-El and Lara sent him to Earth in a small spaceship shortly before Krypton was destroyed in a natural cataclysm. His ship landed in the American country

In [23]:
response = await collector_agent.run("Do we have any hero having name 'Superman' in our Hero server?", ctx=ctx)

print(str(response))

Yes, there is a hero named **Superman** in the Hero server. Here are the details:

- **Name:** Superman
- **Real Name:** Kal-El
- **Origin:** Krypton
- **Team:** Justice League
- **First Appearance:** Action Comics #1, 1938
- **Superpowers:** Superhuman strength, invulnerability, flight, heat vision, x-ray vision, super speed, super breath
- **Biography:** Superman is a superhero created by writer Jerry Siegel and artist Joe Shuster, first appearing in the comic book Action Comics #1, published in America on April 18, 1938. Superman has been regularly published in American comic books since 1938, and has been adapted to other media including radio serials, novels, films, television shows, theater, and video games. Superman was born Kal-El, on the fictional planet Krypton. As a baby, his parents Jor-El and Lara sent him to Earth in a small spaceship shortly before Krypton was destroyed in a natural cataclysm. His ship landed in the American countryside near the fictional town of Smallvi

In [ ]:
response = await collector_agent.run("Đã bao lâu kể từ khi superman xuất hiện ở một đầu truyện?", ctx=ctx)
print(str(response))

In [26]:
response = await collector_agent.run("Xoá toàn bộ hero trong hệ thống", ctx=ctx)
print(str(response))

Tôi đã xóa toàn bộ hero trong hệ thống, bao gồm cả Superman. Nếu bạn cần thêm bất kỳ sự trợ giúp nào khác, hãy cho tôi biết!


In [28]:
response = await collector_agent.run("Giữa 2025-04-01 và 2023-03-01 có bao nhiêu ngày", ctx=ctx)
print(str(response))

Giữa ngày 1 tháng 4 năm 2025 và ngày 1 tháng 3 năm 2023 có tổng cộng **762 ngày**.
